<a href="https://colab.research.google.com/github/muoyo/chicago-ridesharing/blob/master/notebooks/rideshare_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install sodapy

In [0]:
import pandas as pd
import seaborn as sns
from sodapy import Socrata
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
%matplotlib inline


In [131]:
#!/usr/bin/env python

# make sure to install these packages before running:
# pip install pandas
# !pip install sodapy

import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
# client = Socrata("data.cityofchicago.org", None)

# Example authenticated client (needed for non-public datasets):
client = Socrata('data.cityofchicago.org',
                 'Tk6RhuGAFvF9P4ehsysybj3IW',
                 username="mokome@gmail.com",
                 password="Ch1cago!!")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
client.timeout = 10000
results = client.get("m6dm-c72p", limit=200000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)
results_df

,trip_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area,fare,tip,additional_charges,trip_total,shared_trip_authorized,trips_pooled,pickup_centroid_latitude,pickup_centroid_longitude,pickup_centroid_location,dropoff_community_area,dropoff_centroid_latitude,dropoff_centroid_longitude,dropoff_centroid_location,pickup_census_tract,dropoff_census_tract
0,0f40e6c641cf7a0e4fa163e92e75af51aa788017,2019-09-30T23:45:00.000,2019-09-30T23:45:00.000,694,5.195975,17,10,0,2.55,12.55,False,1,41.9465114202,-87.8060200025,"{'type': 'Point', 'coordinates': [-87.80602000...",NaN,NaN,NaN,NaN,NaN,NaN
1,a63f3daa755f4797c8969b80ac6556feaf963bce,2019-09-30T23:45:00.000,2019-09-30T23:45:00.000,21,0.004173,38,2.5,0,2.55,5.05,False,1,41.8129489392,-87.6178596758,"{'type': 'Point', 'coordinates': [-87.61785967...",38,41.8129489392,-87.6178596758,"{'type': 'Point', 'coordinates': [-87.61785967...",NaN,NaN
2,62344555577428006a6c277d5ad9c42b821e7c0f,2019-09-30T23:45:00.000,2019-10-01T00:00:00.000,885,6.99787,24,12.5,0,2.55,15.05,False,1,41.9012069941,-87.6763559892,"{'type': 'Point', 'coordinates': [-87.67635598...",25,41.8906088526,-87.7560467111,"{'type': 'Point', 'coordinates': [-87.75604671...",NaN,NaN
3,1253a740ea7bb6fac28f1c9653f4d37915c6ba6a,2019-09-30T23:45:00.000,2019-09-30T23:45:00.000,655,2.960185,24,7.5,0,2.55,10.05,False,1,41.8997373884,-87.6649539167,"{'type': 'Point', 'coordinates': [-87.66495391...",6,41.936159071,-87.6612652184,"{'type': 'Point', 'coordinates': [-87.66126521...",17031242000,17031062800
4,96389bd840cc2f1d908e92895dcb58f11ac7b06f,2019-09-30T23:45:00.000,2019-10-01T00:00:00.000,768,6.938058,32,10,0,2.55,12.55,False,1,41.8788655841,-87.6251921424,"{'type': 'Point', 'coordinates': [-87.62519214...",3,41.96581197,-87.6558787862,"{'type': 'Point', 'coordinates': [-87.65587878...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,9c8627c355ecea4a8ca2fec2056afacec4a71f9e,2019-09-30T06:00:00.000,2019-09-30T06:15:00.000,881,2.812404,24,7.5,0,2.55,10.05,False,1,41.8996701799,-87.6698377982,"{'type': 'Point', 'coordinates': [-87.66983779...",8,41.8950334495,-87.6197106717,"{'type': 'Point', 'coordinates': [-87.61971067...",17031242100,17031081401
199996,056dd80d6cece5e76e9c295cb0b6bc47d98459e0,2019-09-30T06:00:00.000,2019-09-30T06:45:00.000,2906,17.164743,7,30,0,7.55,37.55,False,1,41.9217783562,-87.6414597588,"{'type': 'Point', 'coordinates': [-87.64145975...",76,41.9790708201,-87.9030396611,"{'type': 'Point', 'coordinates': [-87.90303966...",17031071300,17031980000
199997,0cae9318b78d85bd8b70e59e56e4434efac8fcfe,2019-09-30T06:00:00.000,2019-09-30T06:30:00.000,1860,14.469839,7,22.5,0,7.55,30.05,False,1,41.9217783562,-87.6414597588,"{'type': 'Point', 'coordinates': [-87.64145975...",56,41.785998518,-87.7509342894,"{'type': 'Point', 'coordinates': [-87.75093428...",17031071300,17031980100
199998,ae7b93a906f1124897adbace8d3f54e041baa25a,2019-09-30T06:00:00.000,2019-09-30T06:15:00.000,396,1.545343,7,5,0,2.55,7.55,False,1,41.9305785697,-87.6422063127,"{'type': 'Point', 'coordinates': [-87.64220631...",7,41.9217781876,-87.6510618838,"{'type': 'Point', 'coordinates': [-87.65106188...",17031070102,17031071100


In [132]:
results_df.head()

,trip_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area,fare,tip,additional_charges,trip_total,shared_trip_authorized,trips_pooled,pickup_centroid_latitude,pickup_centroid_longitude,pickup_centroid_location,dropoff_community_area,dropoff_centroid_latitude,dropoff_centroid_longitude,dropoff_centroid_location,pickup_census_tract,dropoff_census_tract
0,0f40e6c641cf7a0e4fa163e92e75af51aa788017,2019-09-30T23:45:00.000,2019-09-30T23:45:00.000,694,5.195975,17,10,0,2.55,12.55,False,1,41.9465114202,-87.8060200025,"{'type': 'Point', 'coordinates': [-87.80602000...",NaN,NaN,NaN,NaN,NaN,NaN
1,a63f3daa755f4797c8969b80ac6556feaf963bce,2019-09-30T23:45:00.000,2019-09-30T23:45:00.000,21,0.004173,38,2.5,0,2.55,5.05,False,1,41.8129489392,-87.6178596758,"{'type': 'Point', 'coordinates': [-87.61785967...",38,41.8129489392,-87.6178596758,"{'type': 'Point', 'coordinates': [-87.61785967...",NaN,NaN
2,62344555577428006a6c277d5ad9c42b821e7c0f,2019-09-30T23:45:00.000,2019-10-01T00:00:00.000,885,6.99787,24,12.5,0,2.55,15.05,False,1,41.9012069941,-87.6763559892,"{'type': 'Point', 'coordinates': [-87.67635598...",25,41.8906088526,-87.7560467111,"{'type': 'Point', 'coordinates': [-87.75604671...",NaN,NaN
3,1253a740ea7bb6fac28f1c9653f4d37915c6ba6a,2019-09-30T23:45:00.000,2019-09-30T23:45:00.000,655,2.960185,24,7.5,0,2.55,10.05,False,1,41.8997373884,-87.6649539167,"{'type': 'Point', 'coordinates': [-87.66495391...",6,41.936159071,-87.6612652184,"{'type': 'Point', 'coordinates': [-87.66126521...",17031242000,17031062800
4,96389bd840cc2f1d908e92895dcb58f11ac7b06f,2019-09-30T23:45:00.000,2019-10-01T00:00:00.000,768,6.938058,32,10,0,2.55,12.55,False,1,41.8788655841,-87.6251921424,"{'type': 'Point', 'coordinates': [-87.62519214...",3,41.96581197,-87.6558787862,"{'type': 'Point', 'coordinates': [-87.65587878...",NaN,NaN


In [0]:
results_df.columns

Index(['trip_id', 'trip_start_timestamp', 'trip_end_timestamp', 'trip_seconds',
       'trip_miles', 'pickup_community_area', 'dropoff_community_area', 'fare',
       'tip', 'additional_charges', 'trip_total', 'shared_trip_authorized',
       'trips_pooled', 'pickup_centroid_latitude', 'pickup_centroid_longitude',
       'pickup_centroid_location', 'dropoff_centroid_latitude',
       'dropoff_centroid_longitude', 'dropoff_centroid_location',
       'pickup_census_tract', 'dropoff_census_tract'],
      dtype='object')

In [133]:
columns_to_use = ['trip_id', 'trip_start_timestamp', 'trip_end_timestamp', 'trip_seconds',
       'trip_miles', 'pickup_community_area', 'fare', 'tip',
       'additional_charges', 'trip_total' ]

columns_to_drop = [ col for col in results_df.columns if col not in columns_to_use ]
df = results_df.drop(columns=columns_to_drop)

df['trip_start_timestamp'] = pd.to_datetime(df['trip_start_timestamp'])
df['trip_end_timestamp'] = pd.to_datetime(df['trip_end_timestamp'])
df['trip_seconds'] = df['trip_seconds'].astype('int64')

for col in ['trip_miles', 'fare', 'tip', 'additional_charges', 'trip_total']:
    df[col] = df[col].astype(float) 

df['start_weekday'] = df['trip_start_timestamp'].apply(lambda d: d.weekday())
df['start_hour'] = df['trip_start_timestamp'].apply(lambda d: d.hour)
df['start_time_block'] = df['start_hour'] // 3

df.head()

,trip_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area,fare,tip,additional_charges,trip_total,start_weekday,start_hour,start_time_block
0,0f40e6c641cf7a0e4fa163e92e75af51aa788017,2019-09-30 23:45:00,2019-09-30 23:45:00,694,5.195975,17,10.0,0.0,2.55,12.55,0,23,7
1,a63f3daa755f4797c8969b80ac6556feaf963bce,2019-09-30 23:45:00,2019-09-30 23:45:00,21,0.004173,38,2.5,0.0,2.55,5.05,0,23,7
2,62344555577428006a6c277d5ad9c42b821e7c0f,2019-09-30 23:45:00,2019-10-01 00:00:00,885,6.997870,24,12.5,0.0,2.55,15.05,0,23,7
3,1253a740ea7bb6fac28f1c9653f4d37915c6ba6a,2019-09-30 23:45:00,2019-09-30 23:45:00,655,2.960185,24,7.5,0.0,2.55,10.05,0,23,7
4,96389bd840cc2f1d908e92895dcb58f11ac7b06f,2019-09-30 23:45:00,2019-10-01 00:00:00,768,6.938058,32,10.0,0.0,2.55,12.55,0,23,7


In [0]:
# public_schools_per_county = df_public_schools['COUNTYFIPS'].value_counts().to_frame()
# public_schools_per_county.reset_index(level=0, inplace=True)
# public_schools_per_county.rename(index=str, columns={"index": "FIPS", "COUNTYFIPS": "NumberOfPublicSchools"}, inplace=True)
# public_schools_per_county_dcmetro = public_schools_per_county.loc[public_schools_per_county['FIPS'].isin(dcmetro_FIPS.values())]

# private_schools_per_county = df_private_schools['COUNTYFIPS'].value_counts().to_frame()
# private_schools_per_county.reset_index(level=0, inplace=True)
# private_schools_per_county.rename(index=str, columns={"index": "FIPS", "COUNTYFIPS": "NumberOfPrivateSchools"}, inplace=True)
# private_schools_per_county_dcmetro = private_schools_per_county.loc[private_schools_per_county['FIPS'].isin(dcmetro_FIPS.values())]

# hospitals_per_county = df_hospitals['COUNTYFIPS'].value_counts().to_frame()
# hospitals_per_county.reset_index(level=0, inplace=True)
# hospitals_per_county.rename(index=str, columns={"index": "FIPS", "COUNTYFIPS": "NumberOfHospitals"}, inplace=True)
# hospitals_per_county['FIPS'] = pd.to_numeric(hospitals_per_county['FIPS'], errors='coerce')
# hospitals_per_county_dcmetro = hospitals_per_county.loc[hospitals_per_county['FIPS'].isin(dcmetro_FIPS.values())]

# df_dcmetro_merged = df_dcmetro.merge(public_schools_per_county_dcmetro, how='left', on='FIPS').merge(
#                                     private_schools_per_county_dcmetro, how='left', on='FIPS').merge(
#                                     hospitals_per_county_dcmetro, how='left', on='FIPS')

In [0]:

weather_df.loc[(weather_df['hour'] < 0) | (weather_df['hour'] > 23)]
#weather_df = weather_df.drop(labels=[96,8832], axis=0)
weather_df['hour'] = weather_df['hour'].apply(lambda x: '{:02d}'.format(x))

In [161]:
weather_df['timestamp'] = pd.to_datetime(weather_df['date'] + ' ' + weather_df['hour'] + ':00:00')
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10223 entries, 0 to 10224
Data columns (total 22 columns):
date                   10223 non-null object
hour                   10223 non-null object
time                   10223 non-null int64
summary                10223 non-null object
icon                   10223 non-null object
precipIntensity        10223 non-null float64
precipProbability      10223 non-null float64
precipType             7033 non-null object
temperature            10223 non-null float64
apparentTemperature    10223 non-null float64
dewPoint               10223 non-null float64
humidity               10223 non-null float64
pressure               10223 non-null float64
windSpeed              10223 non-null float64
windGust               10223 non-null float64
windBearing            10223 non-null int64
cloudCover             10223 non-null float64
uvIndex                10223 non-null int64
visibility             10223 non-null float64
ozone                  10223 

In [162]:
weather_df['timestamp'].date

AttributeError: ignored

In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 14 columns):
trip_id                   200000 non-null object
trip_start_timestamp      200000 non-null datetime64[ns]
trip_end_timestamp        200000 non-null datetime64[ns]
trip_seconds              200000 non-null int64
trip_miles                200000 non-null float64
pickup_community_area     185706 non-null object
dropoff_community_area    183628 non-null object
fare                      200000 non-null float64
tip                       200000 non-null float64
additional_charges        200000 non-null float64
trip_total                200000 non-null float64
start_weekday             200000 non-null int64
start_hour                200000 non-null int64
start_time_block          200000 non-null int64
dtypes: datetime64[ns](2), float64(5), int64(4), object(3)
memory usage: 21.4+ MB


In [134]:
columns_to_use = ['start_weekday', 'start_hour', 'pickup_community_area']

columns_to_drop = [ col for col in df.columns if col not in columns_to_use ]
X_cat = df.drop(columns=columns_to_drop)
X_cat = X_cat.fillna('0')
X_cat

,pickup_community_area,start_weekday,start_hour
0,17,0,23
1,38,0,23
2,24,0,23
3,24,0,23
4,32,0,23
...,...,...,...
199995,24,0,6
199996,7,0,6
199997,7,0,6
199998,7,0,6


In [135]:
enc = OneHotEncoder()
# X_cat_ohe = enc.fit_transform(X_cat)
# columns = enc.get_feature_names(input_features=X_cat.columns)
# X_cat_df = pd.DataFrame(X_cat_ohe.todense(), columns=columns)
y = df['trip_total']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_cat, y, test_size=0.25, random_state=12)

# Transform training and test sets
X_train_enc = enc.fit_transform(X_train, y_train)
X_test_enc = enc.transform(X_test)

# Convert these columns into a DataFrame 
columns = enc.get_feature_names(input_features=X_train.columns)
X_train_df = pd.DataFrame(X_train_enc.todense(), columns=columns, index=X_train.index)
X_test_df = pd.DataFrame(X_test_enc.todense(), columns=columns, index=X_test.index)

X_train_df = sm.add_constant(X_train_df)
model = sm.OLS(y_train,X_train_df).fit()
model.summary()

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             trip_total   R-squared:                       0.313
Model:                            OLS   Adj. R-squared:                  0.313
Method:                 Least Squares   F-statistic:                     726.5
Date:                Wed, 22 Jan 2020   Prob (F-statistic):               0.00
Time:                        05:51:57   Log-Likelihood:            -5.6261e+05
No. Observations:              150000   AIC:                         1.125e+06
Df Residuals:                  149905   BIC:                         1.126e+06
Df Model:                          94                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
pickup_community_area_0     14.3828      0.115    125.247      0.000      14.158      14.608
pickup_community_area_1      0.1299      0.261      0.498      0.618      -0.381       0.641
pickup_community_area_10     0.4940      0.489      1.010      0.312      -0.464       1.452
pickup_community_area_11     0.7820      0.481      1.626      0.104      -0.161       1.725
pickup_community_area_12     1.8125      0.688      2.634      0.008       0.464       3.161
pickup_community_area_13    -0.1988      0.516     -0.385      0.700      -1.210       0.812
pickup_community_area_14    -0.1116      0.333     -0.336      0.737      -0.763       0.540
pickup_community_area_15     0.1072      0.323      0.332      0.740      -0.526       0.740
pickup_community_area_16     0.4128      0.280      1.475      0.140      -0.136       0.961
pickup_community_area_17    -0.7097      0.491     -1.447      0.148      -1.671       0.252
pickup_community_area_18     0.6006      0.794      0.757      0.449      -0.955       2.157
pickup_community_area_19    -0.9583      0.304     -3.158      0.002      -1.553      -0.363
pickup_community_area_2     -0.9504      0.293     -3.240      0.001      -1.525      -0.376
pickup_community_area_20    -1.5918      0.486     -3.274      0.001      -2.545      -0.639
pickup_community_area_21    -0.6997      0.276     -2.532      0.011      -1.241      -0.158
pickup_community_area_22    -0.8666      0.167     -5.188      0.000      -1.194      -0.539
pickup_community_area_23    -1.7094      0.301     -5.674      0.000      -2.300      -1.119
pickup_community_area_24    -1.5514      0.129    -12.011      0.000      -1.805      -1.298
pickup_community_area_25    -1.0891      0.265     -4.105      0.000      -1.609      -0.569
pickup_community_area_26    -2.0092      0.547     -3.675      0.000      -3.081      -0.938
pickup_community_area_27    -1.6353      0.424     -3.861      0.000      -2.465      -0.805
pickup_community_area_28    -1.4288      0.106    -13.473      0.000      -1.637      -1.221
pickup_community_area_29    -1.2903      0.337     -3.834      0.000      -1.950      -0.631
pickup_community_area_3      0.3716      0.198      1.878      0.060      -0.016       0.759
pickup_community_area_30    -0.9607      0.319     -3.007      0.003      -1.587      -0.334
pickup_community_area_31    -1.7359      0.251     -6.912      0.000      -2.228      -1.244
pickup_community_area_32     0.5910      0.102      5.811      0.000       0.392       0.790
pickup_community_area_33     0.0926      0.213      0.435      0.664      -0.325       0.510
pickup_community_area_34    -1.8107      0.414     -4.369      0.000      -2.623      -0.998
pickup_community_area_35    -1.7323      0.313     -5.543      0.000      -2.345      -1.120
pickup_communi

In [0]:
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from sklearn.linear_model import LinearRegression

# Fit the model
linreg = LinearRegression()
linreg.fit(X_train_df, y_train)

# Print R2 and MSE for training and test sets
print('Training r^2:', linreg.score(X_train_df, y_train))
print('Test r^2:', linreg.score(X_test_df, y_test))
print('Training MSE:', mean_squared_error(y_train, linreg.predict(X_train_df)))
print('Test MSE:', mean_squared_error(y_test, linreg.predict(X_test_df)))


Training r^2: 0.3090977429532322
Test r^2: 0.32224597470175154
Training MSE: 108.38359892999804
Test MSE: 103.52141227931178


In [151]:
weather_df = pd.read_csv('https://github.com/muoyo/chicago-ridesharing/raw/master/data/chicago_weather.csv')
weather_df

,date,hour,time,summary,icon,precipIntensity,precipProbability,precipType,temperature,apparentTemperature,dewPoint,humidity,pressure,windSpeed,windGust,windBearing,cloudCover,uvIndex,visibility,ozone,precipAccumulation
0,2018-11-01,0,1541048400,Overcast,cloudy,0.0014,0.05,rain,50.60,50.60,40.06,0.67,1014.7,3.37,4.23,14,0.98,0,10.0,290.8,NaN
1,2018-11-01,1,1541052000,Overcast,cloudy,0.0019,0.06,rain,50.47,50.47,39.10,0.65,1014.1,3.44,5.31,325,1.00,0,10.0,289.5,NaN
2,2018-11-01,2,1541055600,Overcast,cloudy,0.0012,0.05,rain,49.54,48.09,38.92,0.67,1014.1,4.35,7.00,345,1.00,0,10.0,289.1,NaN
3,2018-11-01,3,1541059200,Overcast,cloudy,0.0015,0.05,rain,49.79,47.39,38.98,0.66,1013.8,6.06,11.31,6,1.00,0,10.0,289.0,NaN
4,2018-11-01,4,1541062800,Overcast,cloudy,0.0005,0.04,rain,49.93,46.49,38.13,0.64,1013.3,8.51,13.13,20,0.99,0,10.0,289.1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10220,2019-12-31,19,1577840400,Overcast,cloudy,0.0000,0.00,NaN,27.88,17.97,20.32,0.73,1010.1,11.25,17.40,212,0.91,0,10.0,354.5,NaN
10221,2019-12-31,20,1577844000,Mostly Cloudy,partly-cloudy-night,0.0006,0.01,snow,28.07,16.59,20.64,0.73,1011.4,14.90,17.45,310,0.78,0,10.0,356.8,0.0063
10222,2019-12-31,21,1577847600,Mostly Cloudy,partly-cloudy-night,0.0000,0.00,NaN,27.40,16.31,20.58,0.75,1011.1,13.53,16.62,272,0.64,0,10.0,358.7,NaN
10223,2019-12-31,22,1577851200,Mostly Cloudy,partly-cloudy-night,0.0007,0.01,snow,26.27,16.86,20.83,0.80,1011.2,9.59,15.84,276,0.68,0,10.0,359.4,0.0079
